# exp049
co_visit rank追加

In [1]:
import os
import sys
import traceback
import gc
import random
import pickle
import pathlib
import subprocess
from dataclasses import dataclass
from dotenv import load_dotenv
load_dotenv
sys.path.append(os.getenv('UTILS_PATH'))
from tqdm import tqdm
import multiprocessing
import inspect

import pandas as pd
import numpy as np
import itertools
import cudf
import lightgbm as lgb
import optuna.integration.lightgbm as optuna_lgb
import matplotlib.pyplot as plt
import seaborn as sns
import line_notify
import my_logger
from noglobal import noglobal

# 設定

In [2]:
@dataclass
class Cfg:
    loglevel = "INFO"
    exp_name = "exp049_hypopt"
    seed = 42
    k = 20
    cand_n = 15
    negative_sample = 1
    train_chunk_n = 1
    type2id = {"clicks":0, "carts":1, "orders":2}
    id2type = {0:"clicks", 1:"carts", 2:"orders"}
    train_weeks = ["week3"]
    valid_week = "week4"
    valid_session_n = 100_000
    input_dir = os.getenv('INPUT_DIR')
    output_dir = os.getenv('OUTPUT_DIR')
    prep_dir = os.getenv("PREP_DIR")

cfg = Cfg()
os.makedirs(os.path.join(cfg.output_dir, cfg.exp_name), exist_ok=True)
os.makedirs(os.path.join(cfg.output_dir, cfg.exp_name, "cache"), exist_ok=True)
random.seed(cfg.seed)

logger = my_logger.init_logger(cfg.exp_name)

In [3]:
params = {'objective': 'binary', 
        'boosting': 'gbdt', 
        'learning_rate': 0.1, 
        'metric': 'binary_logloss', 
        'seed': cfg.seed
        }

In [4]:
for type_ in ["clicks", "carts", "orders"]:
    train = pd.read_pickle(cfg.output_dir + f"exp049/cache/{type_}_train.pkl")
    valid = pd.read_pickle(cfg.output_dir + f"exp049/cache/{type_}_valid.pkl")
    target = "labels"
    not_use_cols = ["session", "aid", target]
    features = [c for c in train.columns if c not in not_use_cols]
    
    tr_x, tr_y = train[features], train[target]
    vl_x, vl_y = valid[features], valid[target]
    tr_data = lgb.Dataset(tr_x, label=tr_y)
    vl_data = lgb.Dataset(vl_x, label=vl_y)

    model = optuna_lgb.train(params, tr_data, valid_sets=[tr_data, vl_data],
                             num_boost_round=2000000, early_stopping_rounds=100, verbose_eval=100)
    logger.info(f"{type_} best_params")
    logger.info(model.params)

[I 2022-12-19 10:54:06,211] A new study created in memory with name: no-name-0d5a8659-7cf9-445c-9418-ee9bf20e89cd
feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]/home/kaggler/.local/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/kaggler/.local/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 2634664, number of negative: 2634664
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.445741 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 11905
[LightGBM] [Info] Number of data points in the train set: 5269328, number of used features: 55
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.350707	valid_1's binary_logloss: 0.34536
[200]	valid_0's binary_logloss: 0.348237	valid_1's binary_logloss: 0.343555
[300]	valid_0's binary_logloss: 0.346895	valid_1's binary_logloss: 0.342747
[400]	valid_0's binary_logloss: 0.345826	valid_1's binary_logloss: 0.342236
[500]	valid_0's binary_logloss: 0.344876	valid_1's binary_logloss: 0.341686
[600]	valid_0's binary_logloss: 0.344016	valid

feature_fraction, val_score: 0.339813:  14%|#4        | 1/7 [05:36<33:38, 336.45s/it]

Early stopping, best iteration is:
[2271]	valid_0's binary_logloss: 0.334585	valid_1's binary_logloss: 0.339813
[LightGBM] [Info] Number of positive: 2634664, number of negative: 2634664
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.398425 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 11905
[LightGBM] [Info] Number of data points in the train set: 5269328, number of used features: 55
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.350651	valid_1's binary_logloss: 0.345349
[200]	valid_0's binary_logloss: 0.348261	valid_1's binary_logloss: 0.343633
[300]	valid_0's binary_logloss: 0.346992	valid_1's binary_logloss: 0.342996
[400]	valid_0's binary_logloss: 0.345896	valid_1's binary_logloss: 0.342467
[500]	val

feature_fraction, val_score: 0.339813:  29%|##8       | 2/7 [10:36<26:14, 314.95s/it]

...

Early stopping, best iteration is:
[265]	valid_0's binary_logloss: 0.177191	valid_1's binary_logloss: 0.180768
[LightGBM] [Info] Number of positive: 577974, number of negative: 577974
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.053473 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 12063
[LightGBM] [Info] Number of data points in the train set: 1155948, number of used features: 55
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.189872	valid_1's binary_logloss: 0.181862
[200]	valid_0's binary_logloss: 0.181418	valid_1's binary_logloss: 0.18086
[300]	valid_0's binary_logloss: 0.175312	valid_1's binary_logloss: 0.180778


min_data_in_leaf, val_score: 0.180530:  80%|########  | 4/5 [04:19<01:03, 63.37s/it]

Early stopping, best iteration is:
[275]	valid_0's binary_logloss: 0.176715	valid_1's binary_logloss: 0.180684
[LightGBM] [Info] Number of positive: 577974, number of negative: 577974
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.038833 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 12063
[LightGBM] [Info] Number of data points in the train set: 1155948, number of used features: 55
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.189882	valid_1's binary_logloss: 0.181901
[200]	valid_0's binary_logloss: 0.181487	valid_1's binary_logloss: 0.180804
[300]	valid_0's binary_logloss: 0.175401	valid_1's binary_logloss: 0.180595
[400]	valid_0's binary_logloss: 0.170118	valid_1's binary_logloss: 0.180733


min_data_in_leaf, val_score: 0.180530: 100%|##########| 5/5 [05:26<00:00, 65.23s/it]
8:04:53.971827 	 orders best_params
8:04:53.972905 	 {'objective': 'binary', 'boosting': 'gbdt', 'learning_rate': 0.1, 'metric': 'binary_logloss', 'seed': 42, 'feature_pre_filter': False, 'lambda_l1': 9.356310279757256, 'lambda_l2': 1.3120983078968551e-08, 'num_leaves': 174, 'feature_fraction': 0.5, 'bagging_fraction': 1.0, 'bagging_freq': 0, 'min_child_samples': 20, 'num_iterations': 2000000, 'early_stopping_round': 100}


Early stopping, best iteration is:
[326]	valid_0's binary_logloss: 0.173936	valid_1's binary_logloss: 0.18057
